In [34]:
textRDD = sc.parallelize([
    "10 20 30", 
    "hello 10 there",
    "it's 10 pm"
])

textRDD.collect()

['10 20 30', 'hello 10 there', "it's 10 pm"]

In [53]:
from __future__ import print_function

print(textRDD.count())

print(textRDD.first())

print(textRDD.take(2))

textRDD.foreach(print)

3
10 20 30
['10 20 30', 'hello 10 there']


In [36]:
%%sh 
rm -rf output/sample.txt

In [37]:
textRDD.saveAsTextFile('output/sample.txt')

In [38]:
%%sh 

ls output/sample.txt

cat output/sample.txt/part-*

_SUCCESS
part-00000
part-00001
part-00002
part-00003
10 20 30
hello 10 there
it's 10 pm


In [43]:
textRDD.flatMap(lambda line:line.split())

PythonRDD[97] at RDD at PythonRDD.scala:48

In [60]:
textRDD.collect()

['10 20 30', 'hello 10 there', "it's 10 pm"]

In [66]:
import string
textRDD.flatMap(string.split) \
    .collect()

['10', '20', '30', 'hello', '10', 'there', "it's", '10', 'pm']

In [47]:
import re

def is_number(s):
    return re.match(r'[0-9]+', s)

textRDD.flatMap(lambda line:line.split()) \
    .filter(is_number) \
    .collect()

['10', '20', '30', '10', '10']

In [58]:
textRDD.flatMap(lambda line:line.split()) \
    .filter(is_number) \
    .map(int) \
    .collect()

[10, 20, 30, 10, 10]

In [59]:
import operator
textRDD.flatMap(lambda line:line.split()) \
    .filter(is_number) \
    .map(int) \
    .reduce(operator.add)

80